In [1]:
import pandas as pd
import os

In [60]:
fin_checkv = "quality_summary.tsv"
fin_genomad_virus = "isophage_ragtag_nonRef_virus_summary.tsv"
fin_genomad_plasmid = "isophage_ragtag_nonRef_plasmid_summary.tsv"
fin_iphop = "Host_prediction_to_genome_m90.csv"
fin_vs2 = "final-viral-score.tsv"
fin_vs2_cat = "vs2_category.csv"
fin_clst = "isophage_p95_s95_c95_cluster_ani_raw.tsv"

df_checkv = pd.read_csv(fin_checkv, sep="\t")
df_genomad_virus = pd.read_csv(fin_genomad_virus, sep="\t")
df_genomad_plasmid = pd.read_csv(fin_genomad_plasmid, sep="\t")
df_iphop = pd.read_csv(fin_iphop, sep=",")
df_vs2 = pd.read_csv(fin_vs2, sep="\t")
df_vs2_cat = pd.read_csv(fin_vs2_cat, sep=",")
df_clst = pd.read_csv(fin_clst, sep="\t", header=None, names=["cluster", "seq_name"])

# concat df_genomad_virus and df_genomad_plasmid
df_genomad = pd.concat([df_genomad_virus, df_genomad_plasmid], axis=0)

# sort df_iphop by column "Confidence score" in descending order, and then remove duplicates based on column "Virus" only keep the first one
df_iphop_one = df_iphop.sort_values(by="Confidence score", ascending=False).drop_duplicates(subset="Virus", keep="first")

In [81]:
# merge df_genomad_virus and df_checkv based on seq_name and contig_id
df = df_genomad_virus.merge(df_checkv, left_on="seq_name", right_on="contig_id", how="inner")
df = df.merge(df_vs2, left_on="seq_name", right_on="seqname", how="left")
df = df.merge(df_iphop_one, left_on="seq_name", right_on="Virus", how="left")

# checkv_quality in ["Complete", "High-Quality", "Medium-Quality"]
# df = df[df["checkv_quality"].isin(["Complete", "High-quality", "Medium-quality"])]

In [82]:
vcontigs = [x.replace("isophage_", "") for x in df.seq_name.to_list()]
vcontigs

['DMPmix_3_length=59943_depth=185.62x_circular=true',
 'DMPmix_7_length=9409_depth=0.14x',
 'D45mix_1_length=165254_depth=1.00x_circular=true',
 'D45s11_17_length=40759_depth=57.48x_circular=true',
 'DMPmix_8_length=5416_depth=0.29x',
 'DMPs09_46_length=34615_depth=41.72x',
 'DMPmix_4_length=42642_depth=9.47x_circular=true',
 'D45mix_2_length=42942_depth=0.26x',
 'D45mix_3_length=36732_depth=0.40x',
 'DMPmix_6_length=30335_depth=0.49x',
 'DMPmix_13_length=1117_depth=0.29x']

In [83]:
# cluster in df.seq_name
df_clst2 = df_clst[df_clst["cluster"].isin(vcontigs)]
df_clst2.seq_name.to_list()

['D45mix_1_length=165254_depth=1.00x_circular=true',
 'DMPmix_3_length=59943_depth=185.62x_circular=true',
 'D45mix_2_length=42942_depth=0.26x,DMPmix_10_length=4151_depth=0.15x',
 'DMPmix_4_length=42642_depth=9.47x_circular=true,DMPmix_12_length=1240_depth=0.22x,DMPmix_14_length=1090_depth=0.32x',
 'D45s11_17_length=40759_depth=57.48x_circular=true',
 'D45mix_3_length=36732_depth=0.40x,DMPmix_5_length=35162_depth=4.37x',
 'DMPs09_46_length=34615_depth=41.72x',
 'DMPmix_6_length=30335_depth=0.49x',
 'DMPmix_7_length=9409_depth=0.14x',
 'DMPmix_8_length=5416_depth=0.29x',
 'DMPmix_13_length=1117_depth=0.29x']